到了整个流程的最后一个环节：生成答案。在这一步，大语言模型将下场，根据输入的提示词生成一个输出作为答案。由于模型比较大，在这个项目中使用 Ollama 作为模型服务。Ollama 使用 4bit 量化模型，大幅度减小了资源消耗和占用；同时由于大语言模型性能的提高，小模型就已经可堪大用。所以在这个系列中，我们使用 Qwen 2.5 0.5B 模型。Ollama 的安装方法在系列开篇中已经介绍过了，这里不再赘述。另外本篇还会介绍如何使用 GPT 作为后端语言模型。

In [1]:
import os
from pathlib import Path
from langchain.document_loaders import TextLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')

root = os.getcwd()
loader = TextLoader(os.path.join(root, 'data/paul_graham/paul_graham_essay.txt'))
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(data)
vector_store = FAISS.from_documents(docs, embedding)

query = 'What did the author do growing up?'
res = vector_store.similarity_search(query, k=3)
context = '\n'.join([doc.page_content for doc in res])

/Users/wenjiazhai/miniconda3/envs/langchain/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# 构造提示词
所谓“提示词”就是大语言模型的输入。提示词的质量直接决定了模型生成的文本质量。如何构建大语言模型的提示词是一个重要的研究方向，并且因模型而异，需要不断尝试、调整。一般而言，有以下几个原则：
- 提示词要尽可能清楚，目标明确。
- 提示词内最好有角色分工，比如用户的输入可以是以 "user" 的身份提出的，任务的信息可以在 "system" 中给出，模型生成的答案也可以是 "assistant" 的身份。
- 可以给 AI 立一个人设，高大上的人设在某些时候可能会有正收益，比如“资深记着”、“经验丰富的专家”、“拥有丰富知识的老师”等等。

LangChain 中有关于提示词的功能，现在先手写一个简单的例子。

In [2]:
prompt = f'''
You are a helpful assistant.
Answer the question based on the following context:
{context}
Question: {query}
'''

OK，我们已经把准备工作做好了。然后干嘛呢？下面分别介绍 Ollama 和 GPT 的调用方法。
# Ollama
其实 Ollama 有两个类可以生成文本，一个是 `ChatOllama`，另一个是 `OllamaLLM`。这两个类的参数很像，但是 `ChatOllama` 是一个聊天模型， `OllamaLLM` 是一个文本生成模型。他们相同的主要参数有：
- `model`：模型名称
- `temperature`：温度，默认为 0.8
- `top_k`： 从前 k 个概率最大的词中采样，默认为 40
- `num_predict`：最大输出长度，默认为 128

假设已经载入了 Ollama 模型，下面可以直接推理：

In [3]:
from langchain_ollama.chat_models import ChatOllama
llm = ChatOllama(model="qwen2.5:0.5b")
llm.invoke(prompt)

AIMessage(content='Based on the context provided, the author grew up primarily by working outside of school. They focused on writing and programming during college, which was before they started their career in coding or software development. While they mentioned having essays about various topics written while at university, it seems that these were not published as books yet.\n\nAdditionally, throughout their growing-up years, the author wrote essays and worked on projects related to their interests outside of school. They described writing short stories, and then went on to describe projects such as programming code, doing painting, reading "Hackers & Painters," making dinner parties for friends, and buying a building in Cambridge. These activities clearly demonstrate how they found themselves growing up while still pursuing their academic and personal goals.\n\nThe author\'s life outside of school appears to have been focused on creative pursuits rather than the traditional academi

## `OllamaLLM`

In [4]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model="qwen2.5:0.5b")
llm.invoke(prompt)

"Based on the given context, the author worked outside of school before college by writing and programming stories. These activities were part of their personal interests during the time they weren't enrolled in formal educational institutions or attending universities. The text describes these experiences as a way for the author to help them learn something about programming and storytelling."

可以看到，`ChatOllama` 返回了一个 `AIMessage` 对象，而 `OllamaLLM` 直接返回了一个字符串。前者在处理复杂任务时可以提供更大的可定制性，而后者则更易于使用。
# OpenAI
与 Ollama 相似，langchain_openai 也提供了 `ChatOpenAI` 和 `OpenAI` 类。它们的主要参数有：
- `model`：模型名称
- `temperature`：模型的温度
- `max_tokens`：最大 token 生成数量
- `logprobs`：是否返回概率的对数

由于这个库是一个远程库，还提供了一些客户端参数：
- `api_key`：API 密钥
- `timeout`：请求超时时间
- `base_url`：API 的 URL
- `max_retries`：最大重试次数

限于篇幅，这里只展示 `ChatOpenAI` 类的例子。

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.9, api_key="", model_name="gpt-4o-mini-2024-07-18", max_tokens=256)

In [6]:
print(llm.invoke(prompt))

content='Growing up, the author worked on writing and programming. They wrote short stories, which they acknowledged were not very good, and later wrote essays on various topics. Additionally, they engaged in cooking, hosting dinners for a group of friends, and took on projects like working on spam filters and painting.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 260, 'total_tokens': 319, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None} id='run-166e4384-92f4-4056-8721-8332bf34a98b-0' usage_metadata={'input_tokens': 260, 'output_tokens': 59, 'total_tokens': 319, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'aud